## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Sola,VU,77.97,few clouds,-13.8833,167.5500,Leumerus Bungalows
1,2,Saint Anthony,US,84.34,overcast clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
2,3,Alofi,NU,73.69,broken clouds,-19.0595,-169.9187,Taloa Heights
3,5,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,7,Hilo,US,81.81,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Sola,VU,77.97,few clouds,-13.8833,167.5500,Leumerus Bungalows
1,2,Saint Anthony,US,84.34,overcast clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
2,3,Alofi,NU,73.69,broken clouds,-19.0595,-169.9187,Taloa Heights
3,5,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,7,Hilo,US,81.81,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
5,10,Rikitea,PF,73.96,few clouds,-23.1203,-134.9692,People ThankYou
6,14,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,19,Georgetown,MY,81.90,few clouds,5.4112,100.3354,Cititel Penang
8,20,Thompson,CA,71.51,scattered clouds,55.7435,-97.8558,Burntwood Hotel
9,24,Kavaratti,IN,82.18,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands


In [7]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

348


City_ID                348
City                   348
Country                348
Max Temp               348
Current Description    348
Lat                    348
Lng                    348
Hotel Name             338
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

338


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Sola,VU,77.97,few clouds,-13.8833,167.5500,Leumerus Bungalows
1,2,Saint Anthony,US,84.34,overcast clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
2,3,Alofi,NU,73.69,broken clouds,-19.0595,-169.9187,Taloa Heights
3,5,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,7,Hilo,US,81.81,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sola,VU,77.97,few clouds,-13.8833,167.5500,
1,Saint Anthony,US,84.34,overcast clouds,45.0205,-93.2180,
2,Alofi,NU,73.69,broken clouds,-19.0595,-169.9187,
3,Kapaa,US,82.38,light rain,22.0752,-159.3190,
4,Hilo,US,81.81,scattered clouds,19.7297,-155.0900,
5,Rikitea,PF,73.96,few clouds,-23.1203,-134.9692,
6,Puerto Ayora,EC,73.54,broken clouds,-0.7393,-90.3518,
7,Georgetown,MY,81.90,few clouds,5.4112,100.3354,
8,Thompson,CA,71.51,scattered clouds,55.7435,-97.8558,
9,Kavaratti,IN,82.18,overcast clouds,10.5669,72.6420,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
print(len(clean_hotel_df))
clean_hotel_df.head()

338


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sola,VU,77.97,few clouds,-13.8833,167.5500,Leumerus Bungalows
1,Saint Anthony,US,84.34,overcast clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
2,Alofi,NU,73.69,broken clouds,-19.0595,-169.9187,Taloa Heights
3,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Hilo,US,81.81,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

total 696
-rw-r--r--@ 1 michaelfreel  staff   16772 Aug  5 18:33 Vacation_Search.ipynb
-rw-r--r--  1 michaelfreel  staff   24827 Aug  5 16:03 WeatherPy_vacation.csv
-rw-r--r--@ 1 michaelfreel  staff  300279 Aug  5 16:11 WeatherPy_vacation_map.png
drwxr-xr-x  3 michaelfreel  staff      96 Aug  5 18:31 __pycache__/
-rw-r--r--  1 michaelfreel  staff     118 Jul 31 14:33 config.py
